In [104]:
import time
import random
from collections import deque

In [ ]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            # Dodavanje suseda
            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph


In [ ]:
def bfs_shortest_paths(graf, start):
    dist = {node: float('inf') for node in graf} 
    dist[start] = 0
    queue = deque([start])
    paths = {node: [] for node in graf}
    paths[start] = [start]
    
    while queue:
        current = queue.popleft()
        
        for neighbor in graf[current]:
            if dist[neighbor] == float('inf'):
                dist[neighbor] = dist[current] + 1
                queue.append(neighbor)
            
            if dist[neighbor] == dist[current] + 1:
                paths[neighbor].append(current)
    
    return dist, paths

def calculate_betweenness(graf):
    betweenness = {node: 0 for node in graf}
    
    for start in graf:
        dist, paths = bfs_shortest_paths(graf, start)
        
        for node in graf:
            if node == start:
                continue
            
            for neighbor in paths[node]:
                if neighbor != start:
                    betweenness[neighbor] += 1
    
    return betweenness

def betweenness_centralnost(cvor, betweenness):
    return betweenness.get(cvor, 0) 

In [ ]:
def greedy_burning(graf):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graf.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa
    
    # ako imamo vise kandidata, biramo onog sa najvisim betweenness
    najveci_stepen = max(len(graf[node]) for node in graf)
    kandidati = [node for node in graf if len(graf[node]) == najveci_stepen]
    trenutni_cvor = None
    
    if len(kandidati) > 1:
        betweenness = calculate_betweenness(graf)
        trenutni_cvor = max(kandidati, key=lambda node: betweenness_centralnost(node, betweenness))
    else:
        trenutni_cvor = kandidati[0]
        
    print("pocetni:",trenutni_cvor)
    S.append(trenutni_cvor)
    B.add(trenutni_cvor)
    NB.remove(trenutni_cvor)
    
    
    while len(B)<len(graf):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        novozapaljeni = []
        for cvor in B:
            susjedi = graf[cvor]  # svi susjedi trenutnog zapaljenog cvora
            for susjed in susjedi:
                if susjed in NB:  # ako susjed nije zapaljen
                    novozapaljeni.append(susjed)  # dodajemo ga u zapaljene
                    NB.remove(susjed)  
        
        for nz in novozapaljeni:
            B.add(nz) #dodajemo sve susjede u zapaljene
        
        print("nakon prvog koraka u petlji")
        print("sekvenca>",S)
        print("zapaljeni:",B)
        print("nezapaljeni",NB)
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        buducizapaljeni = []
        for b in B:
            susjedi_b = graf[b]
            for susjed in susjedi_b:
                if susjed in NB:
                    if susjed not in buducizapaljeni:
                        buducizapaljeni.append(susjed)
        print('-----')
        print("buduci:",buducizapaljeni)
        print("br buduci:",len(buducizapaljeni))
        print('-----')
        
        
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_broj_zapaljenih = -1
        najbolji_cvor = None
        kandidati_za_najboljeg = []
        
        if NB:
            for cvor in NB:
                if cvor not in buducizapaljeni: 
                    nezapaljeni_susjedi = len([n for n in graf[cvor] if n not in B and n not in buducizapaljeni])
                    if nezapaljeni_susjedi>max_broj_zapaljenih:
                        max_broj_zapaljenih = nezapaljeni_susjedi
                        
            for cvor in NB:
                if cvor not in buducizapaljeni:
                    nezapaljeni_susjedi = len([n for n in graf[cvor] if n not in B and n not in buducizapaljeni])
                    if nezapaljeni_susjedi == max_broj_zapaljenih:
                        kandidati_za_najboljeg.append(cvor) # u slucaju da imamo vise kandidata za najbolji cvor
                        
            print("kandidati:", kandidati_za_najboljeg)
            
            if len(kandidati_za_najboljeg)>1:# izbor na osnovu betweenness centralnosti
                 najbolji_cvor = max(kandidati_za_najboljeg, key=lambda cvor: betweenness_centralnost(cvor, calculate_betweenness(graf)))
            elif len(kandidati_za_najboljeg)==1:
                najbolji_cvor = kandidati_za_najboljeg[0]
                
            if najbolji_cvor is not None:
                S.append(najbolji_cvor)
                B.add(najbolji_cvor)
                NB.remove(najbolji_cvor)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_cvor = random.choice(list(NB))
                S.append(random_cvor)
                B.add(random_cvor)
                NB.remove(random_cvor)
            print("naj", najbolji_cvor)
        else:
            break
        
        
        
        print("------")
        print("sekvenca>",S)
        print("zapaljeni:",B)
        print("nezapaljeni",NB)
        print("br zapaljenih:",len(B))
        print("br nezapaljenih:",len(NB))
        
        
    return S
        

In [109]:
startTime = time.time()
filename= "graf.mtx"  
#filename = "grafovi/manji/chesapeake.mtx"

graph = load_graph_from_mtx(filename)
burn_order = greedy_burning(graph)

print("Redoslijed paljenja čvorova:", burn_order)
print("Size",len(burn_order))
print("--- %s sekundi ---" % (time.time() - startTime))


pocetni: 1
nakon prvog koraka u petlji
sekvenca> [1]
zapaljeni: {1, 2, 3, 4, 5}
nezapaljeni {6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
-----
buduci: [6, 10, 7, 8, 9]
br buduci: 5
-----
kandidati: [15, 16, 17]
naj 15
------
sekvenca> [1, 15]
zapaljeni: {1, 2, 3, 4, 5, 15}
nezapaljeni {6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17}
br zapaljenih: 6
br nezapaljenih: 11
nakon prvog koraka u petlji
sekvenca> [1, 15]
zapaljeni: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 16}
nezapaljeni {12, 13, 14, 17}
-----
buduci: [12, 13, 14]
br buduci: 3
-----
kandidati: [17]
naj 17
------
sekvenca> [1, 15, 17]
zapaljeni: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 16, 17}
nezapaljeni {12, 13, 14}
br zapaljenih: 14
br nezapaljenih: 3
nakon prvog koraka u petlji
sekvenca> [1, 15, 17]
zapaljeni: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
nezapaljeni set()
-----
buduci: []
br buduci: 0
-----
Redoslijed paljenja čvorova: [1, 15, 17]
Size 3
--- 0.0030007362365722656 sekundi ---
